# Connect to local Elasticsearch
先連上我們在本地電腦架設的Elasticsearch資料庫

若沒有進行任何調整，預設hosts 是 '127.0.0.1', port 在 9200

要架設本地端的Elasticsearch資料庫，只要在./elasticsearch/bin/中於cmd執行   ".\elasticsearch" 即可

In [1]:
from elasticsearch import Elasticsearch

es = Elasticsearch(hosts='127.0.0.1', port=9200)

# Create Index and Set Mappings

先宣告要新建的資料庫 index，以及欄位資料的"型式"、"格式"等等的 mappings 

mappings讓你在新增資料時，會先檢查文件document 對應資料的內容是否符合標準，避免存入不正確的資料

另一個用途則是能進行更有效率地檢索(如text 跟 keyword，雖然都接受string型態的資料，但在檢索的時候text只要有任一字元符合就會回傳，而keyword則必須全部符合才會回傳)



In [2]:
mappings = {
    "settings": {
        "number_of_shards": 2,
        "number_of_replicas": 1
    },
    "mappings":{
        "properties": {
            "id": {
                "type": "integer"
            },
            "first_name": {
                "type": "keyword"
            },
            "last_name": {
                "type": "keyword"
            },
            "email": {
                "type": "keyword"
            }
        }
    }
        
}

es.indices.create(index="test", body=mappings)

D:\anaconda3\envs\pyod\lib\site-packages\elasticsearch\connection\base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.14/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'test'}

# Create Data
利用[Mockaroo](https://mockaroo.com/)生成資料

Elasticsearch.index(index="指定資料庫名稱", id="該筆資料id", body="document內容")

id不給也行，只是我們的資料有自己的id，後續比較好做比對

In [3]:
import csv
file = csv.DictReader(open("MOCK_DATA.csv"))
for row in file:
    doc = {
        "id": row['id'],
        "first_name": row['first_name'],
        "last_name": row['last_name'],
        "email": row['email']
    }
    res = es.index(index='test', id=row['id'], body = doc)

# Get Data
Elasticsearch.get(index='指定資料庫名稱', id='目標id')

In [4]:
res = es.get(index="test", id=599)
print(res['_source'])

{'id': '599', 'first_name': 'Mateo', 'last_name': 'Hebditch', 'email': 'mhebditchgm@ted.com'}


# Search Data

### bool

- must相當於and，符合此區塊下所有條件的資料才會被選中

- should相當於or，符合此區塊下任一條件的資料才會被選中

- must_not跟must功能一樣，只是從選中符合條件的資料變成排除符合條件的資料

- term/terms用於指定條件，並且不會將條件分詞(必須完全符合)，兩者差別僅在條件數量

- match也是用於指定條件，只是會將條件分詞(包含條件中的任一字即可)

### range

- gt Greater than.
- gte Greater than or equal to.
- lt Less than.
- lte Less than or equal to.

### search
透過撰寫query並使用Elasticsearch.search(index="指定資料庫名稱", body="query 內容")

列印res['hits']即可，內容包括：

total：檢索到的目標數

hits：檢索到的目標

In [5]:
# 檢索email為mhebditchgm@ted.com的資料(就是前面id = 599的資料)

query = {
    "query": {
        "bool": {
            "must": [
                { "match": { "email": "mhebditchgm@ted.com" }}
            ]
        }
    }
}

res = es.search(index="test", body=query)
print(res['hits'])

{'total': {'value': 1, 'relation': 'eq'}, 'max_score': 5.7493925, 'hits': [{'_index': 'test', '_type': '_doc', '_id': '599', '_score': 5.7493925, '_source': {'id': '599', 'first_name': 'Mateo', 'last_name': 'Hebditch', 'email': 'mhebditchgm@ted.com'}}]}


In [6]:
# 檢索id>10, <=20的資料
query = {
    "query": {
        "range":{
            "id":{
                "gt":10,
                "lte":20
            }            
        }        
    }
}
res = es.search(index="test", body=query)


In [7]:
# 由前面的範例可觀察到，資料儲存於res["hits"]中的hits key
# 為方便呈現，透過迴圈列印資料
for hit in res["hits"]["hits"]:
    print(hit["_source"])

{'id': '11', 'first_name': 'Enrico', 'last_name': 'Dowd', 'email': 'edowda@blogtalkradio.com'}
{'id': '16', 'first_name': 'Jereme', 'last_name': 'Bayford', 'email': 'jbayfordf@tiny.cc'}
{'id': '18', 'first_name': 'Collette', 'last_name': 'Kinchlea', 'email': 'ckinchleah@theglobeandmail.com'}
{'id': '19', 'first_name': 'Sydney', 'last_name': 'Ebbetts', 'email': 'sebbettsi@marketwatch.com'}
{'id': '12', 'first_name': 'Rae', 'last_name': 'Drakers', 'email': 'rdrakersb@sbwire.com'}
{'id': '13', 'first_name': 'Kirstin', 'last_name': 'Dugall', 'email': 'kdugallc@thetimes.co.uk'}
{'id': '14', 'first_name': 'Towney', 'last_name': 'Beininck', 'email': 'tbeininckd@jigsy.com'}
{'id': '15', 'first_name': 'Maire', 'last_name': 'Forestel', 'email': 'mforestele@posterous.com'}
{'id': '17', 'first_name': 'Janifer', 'last_name': 'Gange', 'email': 'jgangeg@networksolutions.com'}
{'id': '20', 'first_name': 'Haroun', 'last_name': 'Hollingsbee', 'email': 'hhollingsbeej@webnode.com'}


# Search Data by SQL
你也可以透過SQL語法進行查詢

透過 Elasticsearch.sql.query(body={'query':"你的SQL指令"})

結果會回傳在res['row']裡面

In [8]:
sql = """
select count(*) 
from test 
"""

res = es.sql.query(body={'query': sql})
print(res['rows'])

[[1000]]


# Delete document

- delete index :　es.indices.delete(index='test')

- delete document： es.indices.delete(index='test',id='id')

In [9]:
es.indices.delete(index='test')

{'acknowledged': True}